In [19]:
import os
import numpy as np
import torch
from PIL import Image
import cv2
from tqdm import tqdm
import supervision as sv
import pandas as pd
from transformers import AutoImageProcessor, AutoModelForObjectDetection

# Define constants
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "/home/ai-user/Paligemma/detr/RT_DeTr/runs/final"
DATA_DIR = "/home/ai-user/Paligemma/bbox/newthous"
OUTPUT_DIR = "/home/ai-user/Paligemma/detr/RT_DeTr/inference_results"
OUTPUT_CSV_PATH = "/home/ai-user/Paligemma/detr/RT_DeTr/inference_results/predictions_rtdetr.csv"
CLASS_NAMES = ['Normal', 'Fracture']  # Matches subdirectory names
NUM_CLASSES = len(CLASS_NAMES)
CONFIDENCE_THRESHOLD = 0.05  # Further lowered to allow more detections

def load_and_preprocess_image(image_path, processor):
    """Load and preprocess an image for RT-DETR inference."""
    try:
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image_np = np.array(image)
        # Convert BGR to RGB if needed
        if image_np.shape[2] == 3:
            image_np = image_np[:, :, ::-1]
        # Process image with the image processor
        inputs = processor(images=image_np, return_tensors="pt")
        return image, inputs
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None, None

def visualize_predictions(image_path, gt_label, model, processor, output_dir, id2label):
    """Run inference, visualize predictions, and save annotated image."""
    os.makedirs(output_dir, exist_ok=True)
    pil_image, inputs = load_and_preprocess_image(image_path, processor)
    if pil_image is None or inputs is None:
        print(f"Skipping {image_path} due to loading error")
        return "No prediction", "No confidence"
    
    # Move inputs to device
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    
    # Run inference
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Print raw model outputs
    print(f"\nProcessing image: {image_path}")
    print(f"Raw model outputs:")
    print(f"  Logits shape: {outputs.logits.shape}")
    print(f"  Logits: {outputs.logits.cpu().numpy()}")
    print(f"  Predicted boxes shape: {outputs.pred_boxes.shape}")
    print(f"  Predicted boxes: {outputs.pred_boxes.cpu().numpy()}")
    
    # Compute confidence scores from logits
    scores = torch.softmax(outputs.logits, dim=-1).max(dim=-1)[0].cpu().numpy()
    print(f"  Confidence scores (pre-post-processing): {scores}")
    
    # Post-process predictions
    target_sizes = torch.tensor([pil_image.size[::-1]]).to(DEVICE)  # [height, width]
    results = processor.post_process_object_detection(
        outputs, threshold=CONFIDENCE_THRESHOLD, target_sizes=target_sizes
    )[0]
    
    # Print post-processed results
    print(f"Post-processed results:")
    print(f"  Boxes: {results['boxes'].cpu().numpy()}")
    print(f"  Labels: {results['labels'].cpu().numpy()}")
    print(f"  Scores: {results['scores'].cpu().numpy()}")
    
    # Convert to supervision Detections
    detections = sv.Detections(
        xyxy=results['boxes'].cpu().numpy(),
        class_id=results['labels'].cpu().numpy(),
        confidence=results['scores'].cpu().numpy()
    )
    
    # Check if detections are empty
    if len(detections.xyxy) == 0:
        print(f"Warning: No detections for {image_path} with confidence >= {CONFIDENCE_THRESHOLD}")
    
    # Annotate image with bounding boxes
    labels = [
        f"{id2label[class_id]} {confidence:.2f}"
        for class_id, confidence in zip(detections.class_id, detections.confidence)
    ]
    
    annotated_image_np = np.array(pil_image)
    # Use BoxAnnotator with minimal configuration for compatibility
    box_annotator = sv.BoxAnnotator(
        thickness=2,
        color=sv.Color.red()  # Use red for visibility
    )
    annotated_image_np = box_annotator.annotate(
        scene=annotated_image_np,
        detections=detections
    )
    
    # Use LabelAnnotator for labels
    label_annotator = sv.LabelAnnotator(
        color=sv.Color.red()
    )
    annotated_image_np = label_annotator.annotate(
        scene=annotated_image_np,
        detections=detections,
        labels=labels
    )
    
    # Add ground truth label to image
    cv2.putText(
        annotated_image_np,
        f"Ground Truth: {gt_label}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.5,
        (0, 0, 255),
        2,
        cv2.LINE_AA
    )
    
    # Save annotated image
    filename = os.path.basename(image_path)
    save_path = os.path.join(output_dir, filename)
    cv2.imwrite(save_path, cv2.cvtColor(annotated_image_np, cv2.COLOR_RGB2BGR))
    print(f"Saved annotated image to {save_path}")
    
    # Prepare prediction results
    pred_labels = [id2label[class_id] for class_id in detections.class_id]
    confidences = [f"{conf:.2f}" for conf in detections.confidence]
    
    return (
        "; ".join(pred_labels) if pred_labels else "No prediction",
        "; ".join(confidences) if confidences else "No confidence"
    )

def get_image_paths_and_labels(data_dir):
    """Load image paths and their corresponding ground truth labels from subdirectories."""
    image_paths = []
    gt_labels = []
    
    for class_name in CLASS_NAMES:
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.exists(class_dir):
            print(f"Warning: Directory {class_dir} does not exist. Skipping...")
            continue
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(class_dir, img_name))
                gt_labels.append(class_name)
    
    # Shuffle images and labels together
    combined = list(zip(image_paths, gt_labels))
    np.random.shuffle(combined)
    image_paths, gt_labels = zip(*combined) if combined else ([], [])
    
    return list(image_paths), list(gt_labels)

def main():
    # Load image processor and model
    print("Loading RT-DETR model and processor...")
    try:
        processor = AutoImageProcessor.from_pretrained(MODEL_PATH)
        model = AutoModelForObjectDetection.from_pretrained(
            MODEL_PATH,
            num_labels=NUM_CLASSES,
            ignore_mismatched_sizes=True
        )
        model.to(DEVICE)
    except Exception as e:
        print(f"Error loading model or processor: {e}")
        return
    
    # Define id2label mapping (consistent with training)
    id2label = {i: CLASS_NAMES[i] for i in range(NUM_CLASSES)}
    
    # Load image paths and labels from directory
    print(f"Loading images from {DATA_DIR}...")
    image_paths, gt_labels = get_image_paths_and_labels(DATA_DIR)
    
    if not image_paths:
        print(f"No images found in {DATA_DIR}. Exiting...")
        return
    
    print(f"Found {len(image_paths)} images for inference.")
    
    results = []
    
    # Process images
    for img_path, gt_label in tqdm(zip(image_paths, gt_labels), total=len(image_paths), desc="Processing images"):
        prediction, confidence = visualize_predictions(
            img_path, gt_label, model, processor, OUTPUT_DIR, id2label
        )
        results.append({
            'image_name': img_path,
            'ground_truth': gt_label,
            'predictions': prediction,
            'confidence': confidence
        })
    
    # Save results to CSV
    results_df = pd.DataFrame(results)
    os.makedirs(os.path.dirname(OUTPUT_CSV_PATH), exist_ok=True)
    results_df.to_csv(OUTPUT_CSV_PATH, index=False)
    print(f"Predictions saved to {OUTPUT_CSV_PATH}")
    print(f"Annotated images saved to {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

Loading RT-DETR model and processor...


Loading images from /home/ai-user/Paligemma/bbox/newthous...
Found 963 images for inference.


Processing images:   0%|          | 0/963 [00:00<?, ?it/s]


Processing image: /home/ai-user/Paligemma/bbox/newthous/Normal/2025_03_12_24440223_7A1E03C6_EEEF769F.jpeg
Raw model outputs:
  Logits shape: torch.Size([1, 300, 2])
  Logits: [[[-7.202348  -9.439762 ]
  [-6.5972433 -8.634392 ]
  [-7.718758  -9.268696 ]
  [-7.577014  -8.998146 ]
  [-7.6719265 -9.099407 ]
  [-7.1135993 -9.308796 ]
  [-7.1048274 -8.837274 ]
  [-6.3116074 -8.679416 ]
  [-7.1448526 -8.763713 ]
  [-7.621596  -9.164952 ]
  [-7.7380857 -9.502893 ]
  [-7.3225927 -9.0370035]
  [-7.359062  -9.128228 ]
  [-7.223075  -8.760273 ]
  [-7.408363  -9.160225 ]
  [-6.369876  -9.673777 ]
  [-7.462728  -9.202681 ]
  [-6.5280967 -8.646177 ]
  [-6.9761596 -9.155717 ]
  [-7.1505723 -8.697798 ]
  [-7.2662454 -9.3257885]
  [-7.3927655 -8.971735 ]
  [-7.419917  -9.10144  ]
  [-7.373649  -8.871774 ]
  [-7.147759  -9.027254 ]
  [-7.028318  -9.356888 ]
  [-7.180586  -9.259307 ]
  [-7.067431  -9.237716 ]
  [-7.3736925 -9.034832 ]
  [-6.819785  -8.679187 ]
  [-7.0726027 -8.850122 ]
  [-6.738962  -9.1

AttributeError: type object 'Color' has no attribute 'red'

In [21]:
import torch
import requests

import numpy as np
import supervision as sv
import albumentations as A

from PIL import Image
from pprint import pprint
from roboflow import Roboflow
from dataclasses import dataclass, replace
from torch.utils.data import Dataset
from transformers import (
    AutoImageProcessor,
    AutoModelForObjectDetection,
    TrainingArguments,
    Trainer
)
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [22]:
# @title Load model

CHECKPOINT = "PekingU/rtdetr_r50vd_coco_o365"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForObjectDetection.from_pretrained(CHECKPOINT).to(DEVICE)
processor = AutoImageProcessor.from_pretrained(CHECKPOINT)

In [26]:
from PIL import Image
import torch

# Path to the local image file
image_path = "/home/ai-user/Paligemma/detr/RT_DeTr/inference_results/2025_05_03_FEDF1F44_3BFEE365_1F51FDB0.jpeg"

# Open the image from the local file path
image = Image.open(image_path)
inputs = processor(image, return_tensors="pt").to(DEVICE)

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Get image dimensions
w, h = image.size
# Post-process the results
results = processor.post_process_object_detection(
    outputs, target_sizes=[(h, w)], threshold=0.3)

In [33]:
from PIL import Image
import torch
import supervision as sv

# Path to the local image file
image_path = "/home/ai-user/Paligemma/detr/RT_DeTr/inference_results/2025_05_03_FEDF1F44_3BFEE365_1F51FDB0.jpeg"

# Open the image from the local file path
image = Image.open(image_path)
inputs = processor(image, return_tensors="pt").to(DEVICE)

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Get image dimensions
w, h = image.size
# Post-process the results
results = processor.post_process_object_detection(
    outputs, target_sizes=[(h, w)], threshold=0.3
)

# Convert transformer results to supervision Detections
detections = sv.Detections.from_transformers(results[0])

# Extract labels from class IDs
labels = [
    model.config.id2label[class_id]
    for class_id in detections.class_id
]

# Annotate the image
annotated_image = image.copy()
annotated_image = sv.BoxAnnotator().annotate(annotated_image, detections)
annotated_image = sv.LabelAnnotator().annotate(annotated_image, detections, labels=labels)

# Resize the annotated image to a thumbnail (max 600x600)
annotated_image.thumbnail((600, 600))

# Save the annotated image to a file
output_path = "/home/ai-user/Paligemma/detr/RT_DeTr/inference_results/annotated_image.jpeg"
annotated_image.save(output_path)
print(f"Annotated image saved to {output_path}")

Annotated image saved to /home/ai-user/Paligemma/detr/RT_DeTr/inference_results/annotated_image.jpeg


In [ ]:
ds_train = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset.location}/train",
    annotations_path=f"{dataset.location}/train/_annotations.coco.json",
)
ds_valid = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset.location}/valid",
    annotations_path=f"{dataset.location}/valid/_annotations.coco.json",
)
ds_test = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset.location}/test",
    annotations_path=f"{dataset.location}/test/_annotations.coco.json",
)

print(f"Number of training images: {len(ds_train)}")
print(f"Number of validation images: {len(ds_valid)}")
print(f"Number of test images: {len(ds_test)}")

NameError: name 'userdata' is not defined